<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Discussion" data-toc-modified-id="Discussion-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Discussion</a></span><ul class="toc-item"><li><span><a href="#General-notes" data-toc-modified-id="General-notes-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>General notes</a></span></li><li><span><a href="#What-is-done-by-the-moment:" data-toc-modified-id="What-is-done-by-the-moment:-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>What is <strong>done</strong> by the moment:</a></span></li><li><span><a href="#Methodology-related-questions" data-toc-modified-id="Methodology-related-questions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Methodology-related questions</a></span></li><li><span><a href="#Data-related-questions:" data-toc-modified-id="Data-related-questions:-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Data-related questions:</a></span></li></ul></li><li><span><a href="#Headers----technical-section" data-toc-modified-id="Headers----technical-section-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Headers -- technical section</a></span></li><li><span><a href="#A-3-node-test-case" data-toc-modified-id="A-3-node-test-case-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>A 3-node test case</a></span><ul class="toc-item"><li><span><a href="#Case-description" data-toc-modified-id="Case-description-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Case description</a></span></li><li><span><a href="#UC-Model-description" data-toc-modified-id="UC-Model-description-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>UC Model description</a></span></li><li><span><a href="#Model-solution" data-toc-modified-id="Model-solution-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Model solution</a></span></li><li><span><a href="#Solution-analysis----prices" data-toc-modified-id="Solution-analysis----prices-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Solution analysis -- prices</a></span></li></ul></li><li><span><a href="#An-one-node-test-case" data-toc-modified-id="An-one-node-test-case-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>An one-node test case</a></span><ul class="toc-item"><li><span><a href="#Case-description" data-toc-modified-id="Case-description-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Case description</a></span></li><li><span><a href="#Data-structures" data-toc-modified-id="Data-structures-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Data structures</a></span></li><li><span><a href="#Metrics-and-case-description" data-toc-modified-id="Metrics-and-case-description-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Metrics and case description</a></span></li><li><span><a href="#Plain-LMP-case" data-toc-modified-id="Plain-LMP-case-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Plain LMP case</a></span></li><li><span><a href="#IP-and-IP+-approaches" data-toc-modified-id="IP-and-IP+-approaches-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>IP and IP+ approaches</a></span><ul class="toc-item"><li><span><a href="#Out-of-market-corrections-(OMC)----auxiliary-functions" data-toc-modified-id="Out-of-market-corrections-(OMC)----auxiliary-functions-4.5.1"><span class="toc-item-num">4.5.1&nbsp;&nbsp;</span>Out-of-market corrections (OMC) -- auxiliary functions</a></span></li><li><span><a href="#Main-code-for-IP" data-toc-modified-id="Main-code-for-IP-4.5.2"><span class="toc-item-num">4.5.2&nbsp;&nbsp;</span>Main code for IP</a></span></li><li><span><a href="#Main-code-for-IP+" data-toc-modified-id="Main-code-for-IP+-4.5.3"><span class="toc-item-num">4.5.3&nbsp;&nbsp;</span>Main code for IP+</a></span></li></ul></li><li><span><a href="#Dual-Pricing-Algorithm" data-toc-modified-id="Dual-Pricing-Algorithm-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Dual Pricing Algorithm</a></span></li></ul></li></ul></div>

# Comparison of different approaches to LMP setting

## Discussion
### General notes
- it is more or less clear how to "unpack" a network topology
- the IEEE 1888 RTS test case is [here](https://www2.ee.washington.edu/research/pstca/pf118/pg_tca118bus.htm). I.e. the [data](https://www2.ee.washington.edu/research/pstca/pf118/ieee118cdf.txt) are available, as well as [short](https://www2.ee.washington.edu/research/pstca/formats/cdf.txt) and [long](https://ieeexplore.ieee.org/document/4075293/) format descriptions. Note that there are some discrepancies between the two, but we'll manage, they are not terrible.

### What is **done** by the moment:
- a simple [data-parser](./IEEE_CDF_parser.jl), including a helper-function to unpack a Node-incidence matrix;
- a simple model for the 3-node case just to test everything works as expected;
   
### Methodology-related questions

Well, how exactly do we proceed?
- do we allow for multiple agents in the same node?

### Data-related questions:
    - how do we cook up the missing data; most importantly, Pmax, Dmax and utilities? I can imagine we can assume for starters that $P_\max = P_\text{actual}$ from the data file and the same logic for the demand. Maybe later we'll want to assume it is, say 20% more. What do we do with utilities? Is there any common practice for that? or just ensure it is high enough?
    - are we planning to visualize the data somehow? I mean, put numbers on a real network scheme? Is there any standard for that in the field?
---

## Headers -- technical section

In [1]:
using JuMP
using Cbc 
using Clp
using DataFrames
using DataFramesMeta

import IEEE_CDF_parser # my data file parser, includes the NI matrix builder


## A 3-node test case
### Case description

In [2]:
# Cook up come simple test data
# three-bus system
buses = DataFrame(BusNo = [1,2,3], # bus number
                  Pmin = [0,0,0], # min supply
                  Pmax = [10,5,0], # max supply
                  Dmax = [0,0,7], # max demand
                  b = [0,0,50], # consumers' utility per kWh
                  C = [5,3,0], # costs per kWh
                  cSU = [0,0,0]) # startup costs

branches = DataFrame(Arc = ["a","b","c"], TapBusNo = [1, 2, 1], ZBusNo = [2,3,3], x=[0.1,0.1,0.1])


,Arc,TapBusNo,ZBusNo,x
1,a,1,2,0.1
2,b,2,3,0.1
3,c,1,3,0.1


Test case:
![A simple three-node examplez](img/3node.png)

In [3]:
# Make a node-arc incidence matrix

NIM = IEEE_CDF_parser.makeNIMatrix(size(buses,1), branches)

3×3 Array{Float64,2}:
 -1.0   0.0  -1.0
  1.0  -1.0   0.0
  0.0   1.0   1.0

### UC Model description

In [4]:
## ================== OPF model definition =====================
UC = JuMP.Model(solver=CbcSolver())
Nbus = size(buses,1)
Nbranches = size(branches,1)

# constants
const Fmax = 100
Pmax = maximum(buses[:,:Pmax])
Dmax = maximum(buses[:,:Dmax])

@variable(UC, -Fmax <= f[l=1:Nbranches] <= Fmax) # flow through each branch l \in Branches
@variable(UC, 0 <= p[i=1:Nbus] <= Pmax, Cont) # production
@variable(UC, 0<= d[i=1:Nbus] <= Dmax) # consumption
@variable(UC, -2*pi <= θ[i=1:Nbus] <= 2*pi) # phase angle
@variable(UC, z[i=1:Nbus], Bin) # generators' commitment (binary)

@constraintref flows[1:Nbus]

for i=1:Nbus
    flows[i] = @constraint(UC, p[i] + sum(NIM[i,l] * f[l] for l=1:Nbranches) == d[i]) # flow constraints
    @constraint(UC, p[i] >= z[i]*buses[i,:Pmin]) # min production
    @constraint(UC, p[i] <= z[i]*buses[i,:Pmax]) # max production

    @constraint(UC, d[i] <= buses[i, :Dmax])
end

for i=1:Nbranches
    @constraint(UC, f[i] == (1/branches[i,:x])*sum(-NIM[j,i]*θ[j] for j=1:Nbus)) # line flow expression, DC-approximation
end

@constraint(UC, θ[1] == 0) # slack node


@objective(UC, :Max, sum(buses[i,:b]*d[i] for i=1:Nbus) - sum(buses[j,:C]*p[j]+buses[j,:cSU]*z[j] for j=1:Nbus))

UC

Maximization problem with:
 * 16 linear constraints
 * 15 variables: 3 binary
Solver is CbcMathProg

### Model solution

In [5]:
status = solve(UC)
println("Objective value = ", getobjectivevalue(UC))

println("Actual demand = ", getvalue(d))
println("Supply = ", getvalue(p))
println("Commitment flag = ", getvalue(z))
println("theta = ", getvalue(θ))
println("flows = ", getvalue(f))

Objective value = 325.0
Actual demand = [0.0, 0.0, 7.0]
Supply = [2.0, 5.0, 0.0]
Commitment flag = [1.0, 1.0, 0.0]
theta = [0.0, 0.1, -0.3]
flows = [-1.0, 4.0, 3.0]


### Solution analysis -- prices
Now, let us fix the optimal commitments ($z$ 's) -- relax the integrality constraint and obtain prices in nodes.

In [6]:
z_opt = getvalue(z)

3-element Array{Float64,1}:
 1.0
 1.0
 0.0

In [7]:
## ================== OPF model definition =====================
UClp = JuMP.Model(solver=ClpSolver())
Nbus = size(buses,1)
Nbranches = size(branches,1)

# constants
const Fmax = 100
Pmax = maximum(buses[:,:Pmax])
Dmax = maximum(buses[:,:Dmax])

@variable(UClp, -Fmax <= f[l=1:Nbranches] <= Fmax) # flow through each branch l \in Branches
@variable(UClp, 0 <= p[i=1:Nbus] <= Pmax, Cont) # production
@variable(UClp, 0<= d[i=1:Nbus] <= Dmax) # consumption
@variable(UClp, -2*pi <= θ[i=1:Nbus] <= 2*pi) # phase angle
@variable(UClp, 0<= z[i=1:Nbus]<=1) # generators' commitment (not binary anymore)

@constraintref flows[1:Nbus]

for i=1:Nbus
    flows[i] = @constraint(UClp, p[i] + sum(NIM[i,l] * f[l] for l=1:Nbranches) == d[i]) # flow constraints
    @constraint(UClp, p[i] >= z[i]*buses[i,:Pmin]) # min production
    @constraint(UClp, p[i] <= z[i]*buses[i,:Pmax]) # max production
    @constraint(UClp, d[i] <= buses[i, :Dmax])
    @constraint(UClp, z[i] == z_opt[i])
end

for i=1:Nbranches
    @constraint(UClp, f[i] == (1/branches[i,:x])*sum(-NIM[j,i]*θ[j] for j=1:Nbus)) # line flow expression, DC-approximation
end

@constraint(UClp, θ[1] == 0) # slack node


@objective(UClp, :Max, sum(buses[i,:b]*d[i] for i=1:Nbus) - sum(buses[j,:C]*p[j]+buses[j,:cSU]*z[j] for j=1:Nbus))

UClp

Maximization problem with:
 * 19 linear constraints
 * 15 variables
Solver is ClpMathProg

In [8]:
status = solve(UClp)
println("Objective value = ", getobjectivevalue(UC))

println("Actual demand = ", getvalue(d))
println("Supply = ", getvalue(p))
println("Commitment flag = ", getvalue(z))
println("theta = ", getvalue(θ))
println("flows = ", getvalue(f))

println("Prices = ", -getdual(flows))

Objective value = 325.0
Actual demand = [0.0, 0.0, 7.0]
Supply = [2.0, 5.0, 0.0]
Commitment flag = [1.0, 1.0, 0.0]
theta = [-0.0, 0.1, -0.3]
flows = [-1.0, 4.0, 3.0]
Prices = [5.0, 5.0, 5.0]


## An one-node test case
Porting the code from Python

### Case description

In [9]:
# Generators
Gnames = ["A","B"];
cSU = [500;500];
Pmin = [0;10]; 
Pmax = [40;200];
c = [40,60];
Ngen = size(Gnames,1)

# Consumers
Cnames = ["1","2"];
D = [0,1]; 
b = [100,61];
Dmax = [100,30];
Ncons = size(Cnames,1);

### Data structures

In [10]:
macro fmtDF()
    return :( DataFrame(model=String[],  # model name
                        aType = String[], # agent type G / C
                        aName = String[], # agent name
                        vol = Float64[],  # volumes, in kWh (production or consumption)
                        price = Float64[],# price per kWh (if paid / received)
                        lumpSumToAgent = Float64[],  # lump sum payment to the agent, $
                        Profit = Float64[])) # profit (utility) of the agent, $
end
modelOutputs = @fmtDF()

,model,aType,aName,vol,price,lumpSumToAgent,Profit


### Metrics and case description

In [11]:
## some helper functions to produce summary statistics on the case will be there
function describeCase(modelName, modelOut, b,c,cSU)
    Consumers = @where(modelOut, .|(:aType .== "C"));
    Generators = @where(modelOut, .|(:aType .== "G"));
    
    PaidByC = sum(Consumers[:vol] .* Consumers[:price] - Consumers[:lumpSumToAgent])
    PaidToG = sum(Generators[:vol] .* Generators[:price] + Generators[:lumpSumToAgent])
    
    FinBalance = PaidByC - PaidToG;
    
    Utility = sum(Consumers[:vol] .* b)
    
    Costs = sum(Generators[:vol] .* c)
    for i=1:size(Generators,1)
        Costs += (Generators[:vol][i] > 0 ? cSU[i] : 0)
    end
   
    Surplus = Utility - Costs;
    
    return (DataFrame(Approach=modelName, Balance = FinBalance, PaidByC = PaidByC, PaidToG = PaidToG, Surplus=Surplus, Utility=Utility, Costs=Costs));
end

describeCase (generic function with 1 method)

### Plain LMP case

In [13]:
## ================== OPF model definition =====================
UC_plain = JuMP.Model(solver=CbcSolver())

@variable(UC_plain, p[i=1:Ngen]>=0) # production
@variable(UC_plain, 0<= z[i=1:Ngen]<=1,Bin) # commitment 

@variable(UC_plain, d[i=1:Ncons] >= 0, Cont) # consumption

for i=1:Ngen
    @constraint(UC_plain, p[i] >= Pmin[i]*z[i]) # min production
    @constraint(UC_plain, p[i] <= Pmax[i]*z[i]) # max production
end

for j=1:Ncons
    @constraint(UC_plain, d[j] <= Dmax[j])
end

@constraint(UC_plain, Balance, sum(p[i] for i=1:Ngen) == sum(d[j] for j=1:Ncons))

@objective(UC_plain, :Max, sum(b[i]*d[i] for i=1:Ncons) - sum(c[j]*p[j]+cSU[j]*z[j] for j=1:Ngen))

UC_plain

Maximization problem with:
 * 7 linear constraints
 * 6 variables: 2 binary
Solver is CbcMathProg

In [14]:
status = solve(UC_plain)

:Optimal

In [42]:
z_opt = getvalue(z)

UC_plain_rel = JuMP.Model(solver=ClpSolver())

@variable(UC_plain_rel, p[i=1:Ngen]>=0) # production
@variable(UC_plain_rel, 0<= z[i=1:Ngen]<=1,Cont) # commitment 

@variable(UC_plain_rel, d[i=1:Ncons] >= 0, Cont) # consumption

@constraint(UC_plain_rel, PminCons[i=1:Ngen], p[i] >= Pmin[i]*z[i]) # min production
@constraint(UC_plain_rel, PmaxCons[i=1:Ngen], p[i] <= Pmax[i]*z[i]) # max production

@constraint(UC_plain_rel, RelaxZ[i=1:Ngen], z[i] == z_opt[i])

@constraint(UC_plain_rel, DmaxCons[j=1:Ncons], d[j] <= Dmax[j])

@constraint(UC_plain_rel, Balance, sum(p[i] for i=1:Ngen) == sum(d[j] for j=1:Ncons))

@objective(UC_plain_rel, :Max, sum(b[i]*d[i] for i=1:Ncons) - sum(c[j]*p[j]+cSU[j]*z[j] for j=1:Ngen))

status = solve(UC_plain_rel)
UC_plain_rel

Maximization problem with:
 * 9 linear constraints
 * 6 variables
Solver is ClpMathProg

In [16]:
out = @fmtDF

price = -getdual(Balance)

for i=1:Ngen
            push!(out,["Plain LMP",
            "G",
            Gnames[i],
            getvalue(p[i]),
            price,
            0,
            getvalue(p[i])*(price - c[i]) - cSU[i]*getvalue(z[i])
            ])
end

for i=1:Ncons
    push!(out,[
            "Plain LMP",
            "C",
            Cnames[i],
            getvalue(d[i]),
            price,
            0,
            getvalue(d[i])*(b[i] - price)
            ])
end

out

,model,aType,aName,vol,price,lumpSumToAgent,Profit
1,Plain LMP,G,A,40.0,60.0,0.0,300.0
2,Plain LMP,G,B,90.0,60.0,0.0,-500.0
3,Plain LMP,C,1,100.0,60.0,0.0,4000.0
4,Plain LMP,C,2,30.0,60.0,0.0,30.0


In [17]:
modelOutputs = out

,model,aType,aName,vol,price,lumpSumToAgent,Profit
1,Plain LMP,G,A,40.0,60.0,0.0,300.0
2,Plain LMP,G,B,90.0,60.0,0.0,-500.0
3,Plain LMP,C,1,100.0,60.0,0.0,4000.0
4,Plain LMP,C,2,30.0,60.0,0.0,30.0


In [18]:
modelMetrics = describeCase("Plain LMP", out,b,c,cSU)

,Approach,Balance,PaidByC,PaidToG,Surplus,Utility,Costs
1,Plain LMP,0.0,7800.0,7800.0,3830.0,11830.0,8000.0


### IP and IP+ approaches

#### Out-of-market corrections (OMC) -- auxiliary functions

In [51]:
## distribution of OMC between consumers
function distributeOMC(FinImbalance, λ, b, volumes)
    # we understand an imbalance >0 when there is something to pay BY consumers
    N = size(volumes,1)
    OMC = DataFrame(index = 1:N, B = b, vol=volumes, val=zeros(N))
    
    if FinImbalance==0
        # just nothing more to do
        return(OMC);
    end
    
    ## we can determine the ``fair'' rate -- but it might be not possible (incentive-incompatible) for everybody
    ## so we start with consumers with the /least/ utility 'margin', i.e. smalles marginal utility
    OMC = sort!(OMC,:B)
    
    currentImbalance = FinImbalance;
    currentVolumes = sum(volumes);
    
    for i=1:N
        OMC_rate = currentImbalance / currentVolumes; ## a rate for Out of Market Correction (per kWh)
        OMC[:val][i] = -minimum([OMC_rate (OMC[:B][i] - λ)])*OMC[:vol][i]; ## min to keep individual rationality (nonnegative profits)
        currentImbalance -= OMC[:val][i]
        currentVolumes -= OMC[:vol][i]
    end

    return (sort!(OMC, :index)); # note that we need to revert to the initial order of consumers
end

distributeOMC (generic function with 1 method)

In [43]:
getdual(RelaxZ)

2-element Array{Float64,1}:
   -0.0
 -500.0

#### Main code for IP

In [53]:
## here we utilize the same problem to produce "IP" pricing (payments)
out = @fmtDF

price = -getdual(Balance);
Gpayments = -getdual(RelaxZ) - getdual(z);

## now, redistribute the imbalance:
imbalance = sum(Gpayments);
println("Market imbalance (=payments to generators): ", imbalance)

Cpayments = distributeOMC(imbalance, price,  b, getvalue(d))[:val]

for i=1:Ngen
            push!(out,["IP",
            "G",
            Gnames[i],
            getvalue(p[i]),
            price,
            Gpayments[i],
            getvalue(p[i])*(price - c[i]) - cSU[i]*getvalue(z[i]) + Gpayments[i]
            ])
end

for i=1:Ncons
    push!(out,[
            "IP",
            "C",
            Cnames[i],
            getvalue(d[i]),
            price,
            Cpayments[i],
            getvalue(d[i])*(b[i] - price)+Cpayments[i]
            ])
end

out

Market imbalance (=payments to generators): 200.0


,model,aType,aName,vol,price,lumpSumToAgent,Profit
1,IP,G,A,40.0,60.0,-300.0,0.0
2,IP,G,B,90.0,60.0,500.0,0.0
3,IP,C,1,100.0,60.0,-230.0,3770.0
4,IP,C,2,30.0,60.0,-30.0,0.0


In [54]:
describeCase("IP",out,b,c,cSU)

,Approach,Balance,PaidByC,PaidToG,Surplus,Utility,Costs
1,IP,60.0,8060.0,8000.0,3830.0,11830.0,8000.0


In [ ]:
modelOutputs = vcat(modelOutputs,out)
modelMetrics = vcat(modelMetrics, describeCase("IP", out,b,c,cSU))

#### Main code for IP+

In [50]:
## here we utilize the same problem to produce "IP+" pricing (payments)
out = @fmtDF

price = -getdual(Balance);
Gpayments = -getdual(RelaxZ);

## now, redistribute the imbalance:
imbalance = sum(Gpayments);
println("Market imbalance (=payments to generators): ", imbalance)

Cpayments = distributeOMC(imbalance, price,  b, getvalue(d))[:val]

for i=1:Ngen
            push!(out,["IP+",
            "G",
            Gnames[i],
            getvalue(p[i]),
            price,
            Gpayments[i],
            getvalue(p[i])*(price - c[i]) - cSU[i]*getvalue(z[i]) + Gpayments[i]
            ])
end

for i=1:Ncons
    push!(out,[
            "IP+",
            "C",
            Cnames[i],
            getvalue(d[i]),
            price,
            Cpayments[i],
            getvalue(d[i])*(b[i] - price)+Cpayments[i]
            ])
end

out

Market imbalance (=payments to generators): 500.0


,model,aType,aName,vol,price,lumpSumToAgent,Profit
1,IP+,G,A,40.0,60.0,0.0,300.0
2,IP+,G,B,90.0,60.0,500.0,0.0
3,IP+,C,1,100.0,60.0,-530.0,3470.0
4,IP+,C,2,30.0,60.0,-30.0,0.0


In [34]:
getdual(z[2])

0.0

### Dual Pricing Algorithm

In [21]:
## first, let us fix optimal volumes
p_opt = getvalue(p);
d_opt = getvalue(d);
# we already have z_opt

In [22]:
UC_DPA = JuMP.Model(solver=ClpSolver())
const M = 100000

@variable(UC_DPA, Up_d[i=1:Ncons] >=0 , Cont) # Uplift TO demand (consumers)
@variable(UC_DPA, Up[i=1:Ngen] >=0, Cont) # Uplift TO generation
@variable(UC_DPA, Upc_d[i=1:Ncons] >=0 , Cont) # additional charge to demand (consumers)
@variable(UC_DPA, Upc[i=1:Ngen] >=0, Cont) # additional charge to generators

@variable(UC_DPA, Ψ[i=1:Ncons] >= -M, Cont) # consumers' utilities
@variable(UC_DPA, Π[j=1:Ngen] >= -M, Cont) # generators' profits

@variable(UC_DPA, -M <= λdpa <= M, Cont)

## objective

@objective(UC_DPA, :Min, sum(d_opt[i]*Up_d[i] for i=1:Ncons)+sum(p_opt[j]*Up[j] for j=1:Ngen))

## constraints
@constraint(UC_DPA, sum(d_opt[i]*(Up_d[i] - Upc_d[i]) for i=1:Ncons)
    == sum(p_opt[j]*(Up[j] - Upc[j]) for j=1:Ngen)) # Balance of payments (no money on the table)

@constraint(UC_DPA, IC_Utility[i=1:Ncons], Ψ[i] == d_opt[i]*(b[i]-λdpa + Up_d[i] - Upc_d[i]) ) # Utility
@constraint(UC_DPA, IC_Profits[j=1:Ngen], Π[j] == p_opt[j]*(λdpa - c[j]+Up[j] - Upc[j]) - cSU[j]*z_opt[j]) # Profits

@constraint(UC_DPA, Ψ[i=1:Ncons] .>= 0 )
@constraint(UC_DPA, Π[i=1:Ngen] .>= 0  )

UC_DPA

Minimization problem with:
 * 9 linear constraints
 * 13 variables
Solver is ClpMathProg

In [23]:
status = solve(UC_DPA)

:Optimal

In [24]:
out = @fmtDF

price = getvalue(λdpa);

for i=1:Ngen
            push!(out,["DPA",
            "G",
            Gnames[i],
            p_opt[i],
            price,
            (getvalue(Up[i]) - getvalue(Upc[i]))*p_opt[i],
            getvalue(Π[i])
            ])
end

for i=1:Ncons
    push!(out,[
            "DPA",
            "C",
            Cnames[i],
            d_opt[i],
            price,
            (getvalue(Up_d[i]) - getvalue(Upc_d[i]))*d_opt[i],
            getvalue(Ψ[i])
            ])
end

out

,model,aType,aName,vol,price,lumpSumToAgent,Profit
1,DPA,G,A,40.0,65.5556,0.0,522.222
2,DPA,G,B,90.0,65.5556,0.0,0.0
3,DPA,C,1,100.0,65.5556,-136.667,3307.78
4,DPA,C,2,30.0,65.5556,136.667,0.0


In [25]:
caseUpd = describeCase("DPA", out,b,c,cSU)

,Approach,Balance,PaidByC,PaidToG,Surplus,Utility,Costs
1,DPA,0.0,8522.22,8522.22,3830.0,11830.0,8000.0


In [26]:
modelMetrics = vcat(modelMetrics, caseUpd)

,Approach,Balance,PaidByC,PaidToG,Surplus,Utility,Costs
1,Plain LMP,0.0,7800.0,7800.0,3830.0,11830.0,8000.0
2,DPA,0.0,8522.22,8522.22,3830.0,11830.0,8000.0
